In [1]:
import sys, os, random, re
import numpy as np

import importlib
sys.path.insert(0, '../src/')
import dataset

In [2]:
special_tokens = {
    'conversation_start': '[CSTART]',
    'conversation_end': '[CEND]',
    'message_start': '[MSTART]',
    'breakpoint': '[BREAK]',
    'message_end': '[MEND]',
    'writes': '[WRITES]',
    'avoid': '[AVOID]',
}

In [3]:
importlib.reload(dataset)

s = dataset.load_simulated_conversations('../data/simulated_conversations')
len(s)

16

In [4]:
importlib.reload(dataset)

m = dataset.load_messenger_conversations('/home/patz/Heap/messages/')
m = [dataset.rename_conversation_user(c, 'Patztablook TwentyTwo', 'you') for c in m]
len(m)

79

In [5]:
importlib.reload(dataset)

url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
m1 = [dataset.mark_avoid(c, url_pattern, special_tokens) for c in m]
s1 = [dataset.mark_avoid(c, url_pattern, special_tokens) for c in s]

In [11]:
importlib.reload(dataset)

def message_filter(content):
    if content is None: return True # empty messages will be used later

    # application message, spam
    content = content.strip()
    blacklist = ['ə', 'ɥ', 'ʇ', 'ʎ', 'ɹ', 'ƃ',
                'se svymi geny? 0=ne', 're you satisfied with ur genes?', 'ould you describe urself as',
                'nyom', "hen you want to retain something in your", "thy weight and height"]
    for b in blacklist:
        if b in content: return False
    
    # wall of text
    if len(content) > 128 or len(content.splitlines()) > 4: return False
    
    # reference to a previous message
    if content in ['.', '*'] : return False

    # fixing a previous message (probably a typo):
    if content[0] == '*' and content[1] != ' ' and len(content.split(' ')) <= 2 and content.count('*') == 1: return False
    if content[-1] == '*' and content[-2] != ' ' and len(content.split(' ')) <= 2 and content.count('*') == 1: return False

    # decrease the frequency of overused phrase messages
    rates = {"ik": 0.7,
             "irl": 0.5,
             "xd": 0.7}
    words = [w for w in re.split(r'(\W)', content) if w.strip()]
    for r in rates:
        if len(words) <= 2 and b in words:
            if np.random.random() < 1 - rates[r]:
                return False
    
    return True

m2 = [dataset.filter_messages_by_content(c, message_filter) for c in m1]

In [12]:
m2 = list(filter(lambda c: len(c) >= 64, m2))
len(m2)

49

In [13]:
names = ['alice', 'bob', 'martin', 'john', 'alex', 'mary', 'filip', 'martha', 'elisa', 'peter', 'paul', 'emil', 'honza', 'cutecat1', 'WIZARD', 'E', '0x01',
         'george', 'jirka', 'aneta', 'owo', 'daniel', 'jaroslav', 'xi', 'william', 'bill', 'harrison']
s2 = [c1 for c in s1 for c1 in dataset.sample_names(c, 'User1', names, 3)]

m2 = [c1 for c in m2 for c1 in dataset.split(c, message_limit=100)]

len(s2), len(m2)

(48, 2694)

In [14]:
val_indices = [] #[-1, -3, -13]
train_indices = np.delete(np.arange(len(m2)), val_indices)

print(len(m2))
for i in val_indices:
    print(m2[i].participants)

2694


In [15]:
importlib.reload(dataset)

train_conversations = [m2[i] for i in train_indices] + s2
train_conversations = np.random.permutation(np.array(train_conversations, dtype=object))

val_conversations = [m2[i] for i in val_indices]

dataset.generate_corpus('../data/train.txt', train_conversations, special_tokens, break_tollerance_s=180)
dataset.generate_corpus('../data/val.txt', val_conversations, special_tokens, break_tollerance_s=180)